In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = saw
band = [11]
commitnumber = de5d43e
molecule = {'co2': 0.0004}
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [3, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.75), (1.75, 1.6, 1.85)]
  wgt = [(0.7, 0.8, 0.7), (0.8, 0.7, 0.8)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.000964                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.079065
0.000750      2.0 -0.155994                  2.0 -0.141184
0.001052      3.0 -0.112857                  3.0 -0.100464
0.001476      4.0 -0.091207                  4.0 -0.080521
0.002070      5.0 -0.074267                  5.0 -0.078299
0.002904      6.0 -0.057905                  6.0 -0.081520
0.004074      7.0 -0.041275                  7.0 -0.081346
0.005714      8.0 -0.024465                  8.0 -0.072103
0.008015      9.0 -0.007901                  9.0 -0.053563
0.011243     10.0  0.008007                 10.0 -0.026942
0.015771     11.0  0.023183                 11.0  0.005190
0.022122     12.0  0.038618                 12.0  0.041933
0.031031     13.0  0.059943                 13.0  0.086252
0.043528     14.0  0.075282                 14.0  0.122028
0.061057     15.0  0.072367                 15.0  0.132535
0.085645     16.0  0.067575                 16.0  0.127627
0.120136     17.0  0.069055                 17.0  0.115765
0.168516     18.0  0.073797                 18.0  0.097924
0.236378     19.0  0.082006                 19.0  0.080830
0.331549     20.0  0.096763                 20.0  0.075700
0.465100     21.0  0.132414                 21.0  0.106339
0.652400     22.0  0.119620                 22.0  0.096388
0.915100     23.0  0.061907                 23.0  0.051056
1.283650     24.0  0.030065                 24.0  0.034725
1.800600     25.0  0.016397                 25.0  0.027466
2.525700     26.0  0.008810                 26.0  0.019325
3.542800     27.0  0.003831                 27.0  0.010470
4.969550     28.0  0.001619                 28.0  0.003298
6.970850     29.0  0.001396                 29.0  0.000383
9.778100     30.0  0.001181                 30.0  0.000691
13.715850    31.0  0.000604                 31.0  0.001384
19.239350    32.0 -0.000120                 32.0  0.000890
26.987250    33.0 -0.000274                 33.0  0.000418
37.855300    34.0  0.000020                 34.0  0.000301
53.100050    35.0  0.000096                 35.0 -0.000004
73.887500    36.0  0.000112                 36.0 -0.000117
97.662500    37.0  0.000127                 37.0 -0.000082
121.437500   38.0 -0.000019                 38.0 -0.000223
145.212500   39.0 -0.000209                 39.0 -0.000408
168.987500   40.0 -0.000322                 40.0 -0.000503
192.762500   41.0 -0.000424                 41.0 -0.000605
216.537500   42.0 -0.000530                 42.0 -0.000724
240.312500   43.0 -0.000652                 43.0 -0.000859
264.087500   44.0 -0.000840                 44.0 -0.001059
287.862500   45.0 -0.001261                 45.0 -0.001530
311.637500   46.0 -0.001548                 46.0 -0.001861
335.412500   47.0 -0.001367                 47.0 -0.001677
359.187500   48.0 -0.001124                 48.0 -0.001458
382.962500   49.0 -0.001020                 49.0 -0.001372
406.737500   50.0 -0.000930                 50.0 -0.001281
430.512500   51.0 -0.000831                 51.0 -0.001142
454.287500   52.0 -0.000715                 52.0 -0.000983
478.062500   53.0 -0.000578                 53.0 -0.000772
501.837500   54.0 -0.000419                 54.0 -0.000534
525.612500   55.0 -0.000236                 55.0 -0.000261
549.387500   56.0 -0.000032                 56.0  0.000044
573.162500   57.0  0.000190                 57.0  0.000376
596.937500   58.0  0.000421                 58.0  0.000724
620.712500   59.0  0.000650                 59.0  0.001049
644.487500   60.0  0.000866                 60.0  0.001384
668.262500   61.0  0.001063                 61.0  0.001655
692.037500   62.0  0.001238                 62.0  0.001939
715.812500   63.0  0.001395                 63.0  0.002180
739.587500   64.0  0.001547                 64.0  0.002412
76

# Fluxes by Layer

CRD                        CLIRAD                    
                        flug      fldg     fnetg      flug      fldg     fnetg
pressure     level                                                            
0.000000e+00 1     -1.267380  0.000000 -1.267380       NaN       NaN       NaN
1.000000e-08 1           NaN       NaN       NaN -1.235448  0.000002 -1.235445
6.244000e-04 2     -1.267395  0.000016 -1.267380 -1.235471  0.000031 -1.235440
8.759000e-04 3     -1.267403  0.000019 -1.267384 -1.235482  0.000038 -1.235444
1.228600e-03 4     -1.267414  0.000025 -1.267389 -1.235497  0.000049 -1.235448
1.723400e-03 5     -1.267429  0.000034 -1.267394 -1.235517  0.000064 -1.235453
2.417400e-03 6     -1.267448  0.000048 -1.267400 -1.235542  0.000083 -1.235459
3.390900e-03 7     -1.267474  0.000066 -1.267407 -1.235576  0.000107 -1.235469
4.756500e-03 8     -1.267507  0.000093 -1.267414 -1.235619  0.000137 -1.235482
6.672000e-03 9     -1.267551  0.000131 -1.267419 -1.235674  0.000176 -1.235498
9.358900e-03 10    -1.267607  0.000185 -1.267422 -1.235745  0.000229 -1.235515
1.312780e-02 11    -1.267678  0.000260 -1.267418 -1.235833  0.000306 -1.235527
1.841450e-02 12    -1.267769  0.000365 -1.267404 -1.235943  0.000419 -1.235524
2.583020e-02 13    -1.267879  0.000509 -1.267370 -1.236076  0.000589 -1.235487
3.623230e-02 14    -1.268002  0.000706 -1.267296 -1.236224  0.000843 -1.235381
5.082340e-02 15    -1.268126  0.000960 -1.267166 -1.236378  0.001208 -1.235170
7.129060e-02 16    -1.268256  0.001266 -1.266990 -1.236546  0.001698 -1.234849
1.000000e-01 17    -1.268401  0.001640 -1.266760 -1.236736  0.002322 -1.234414
1.402710e-01 18    -1.268552  0.002121 -1.266431 -1.236943  0.003080 -1.233862
1.967600e-01 19    -1.268691  0.002753 -1.265937 -1.237151  0.003944 -1.233207
2.759970e-01 20    -1.268767  0.003599 -1.265168 -1.237325  0.004876 -1.232448
3.871000e-01 21    -1.268644  0.004750 -1.263894 -1.237339  0.005887 -1.231452
5.431000e-01 22    -1.267742  0.006295 -1.261447 -1.236530  0.007043 -1.229486
7.617000e-01 23    -1.265939  0.007590 -1.258350 -1.234715  0.007724 -1.226990
1.068500e+00 24    -1.264056  0.007956 -1.256100 -1.232802  0.007668 -1.225135
1.498800e+00 25    -1.262389  0.007822 -1.254567 -1.231094  0.007730 -1.223364
2.102400e+00 26    -1.260919  0.007524 -1.253395 -1.229538  0.008138 -1.221400
2.949000e+00 27    -1.259644  0.007133 -1.252511 -1.228124  0.008662 -1.219462
4.136600e+00 28    -1.258647  0.006674 -1.251972 -1.226933  0.008944 -1.217989
5.802500e+00 29    -1.257933  0.006280 -1.251653 -1.226007  0.008669 -1.217338
8.139200e+00 30    -1.257358  0.006091 -1.251267 -1.225205  0.007973 -1.217232
1.141700e+01 31    -1.256854  0.006046 -1.250808 -1.224451  0.007487 -1.216964
1.601470e+01 32    -1.256505  0.006026 -1.250479 -1.223878  0.007668 -1.216210
2.246400e+01 33    -1.256601  0.006030 -1.250571 -1.223890  0.008360 -1.215530
3.151050e+01 34    -1.257282  0.006417 -1.250865 -1.224636  0.009555 -1.215082
4.420010e+01 35    -1.258340  0.007506 -1.250834 -1.225832  0.011202 -1.214630
6.200000e+01 36    -1.259756  0.009124 -1.250632 -1.227463  0.012824 -1.214638
8.577500e+01 37    -1.261431  0.011115 -1.250316 -1.229375  0.014407 -1.214969
1.095500e+02 38    -1.262856  0.012898 -1.249958 -1.231027  0.015826 -1.215201
1.333250e+02 39    -1.264204  0.014192 -1.250011 -1.232644  0.016816 -1.215828
1.571000e+02 40    -1.265712  0.015111 -1.250601 -1.234542  0.017563 -1.216978
1.808750e+02 41    -1.267403  0.015896 -1.251507 -1.236745  0.018351 -1.218394
2.046500e+02 42    -1.269294  0.016592 -1.252703 -1.239276  0.019178 -1.220098
2.284250e+02 43    -1.271416  0.017220 -1.254196 -1.242174  0.020037 -1.222137
2.522000e+02 44    -1.273827  0.017796 -1.256032 -1.245473  0.020916 -1.224557
2.759750e+02 45    -1.276731  0.018334 -1.258397 -1.249352  0.021811 -1.227540
2.997500e+02 46    -1.280917  0.018969 -1.261947 -1.254712  0.022861 -1.231851
3.235250e+02 47    -1.286861  0.020553 -1.266308 -1.262116  0.025023 -1.237

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')